## Dramatron

<img src="./dramatron.png" />

Implementation of [Dramatron](https://arxiv.org/pdf/2209.14958.pdf), Deepmind's generative story prompt model, which starts with a logline and progressively builds elements of a story ending with dialogue for each scene.

In [1]:
from promptx import load

load()

2023-10-30 02:48:21.419 | INFO     | promptx:load:137 - loading local app from /home/rjl/promptx/scratch/dramatron-redo
2023-10-30 02:48:21.422 | INFO     | promptx:load:140 - loaded environment variables from /home/rjl/promptx/scratch/dramatron-redo/.env
2023-10-30 02:48:21.423 | INFO     | promptx:load:141 - API KEY CQZm7


<App local path=/home/rjl/promptx/scratch/dramatron-redo>

First define the pydantic types used to generate the story.

In [2]:
from typing import List
from pydantic import Field

from promptx import prompt, store, query
from promptx.collection import Entity


class Character(Entity):
    name: str
    description: str


class Location(Entity):
    name: str
    description: str


class SceneBeat(Entity):
    location: str
    plot_element: str
    description: str


class Story(Entity):
    logline: str
    title: str = None
    outline: List[SceneBeat] = None
    characters: List[Character] = None
    locations: List[Location] = None

Before generating a story lets define some examples to use as few shots in the prompts. We'll use the Star Wars examples defined in the paper.

In [3]:
star_wars = Story(
    title="Star Wars",
    logline='''
    A science - fiction fantasy about a naive but ambitious farm boy from a 
    backwater desert who discovers powers he never knew he had when he teams 
    up with a feisty princess, a mercenary space pilot and an old wizard warrior 
    to lead a ragtag rebellion against the sinister forces of the evil Galactic 
    Empire.
    ''',
    characters=[
        Character(
            name='Luke Skywalker',
            description='''
            Luke Skywalker is the hero. A naive farm boy, he will 
            discover special powers under the guidance of mentor 
            Ben Kenobi.
            ''',
        ),
        Character(
            name='Ben Kenobi',
            description='''
            Ben Kenobi is the mentor figure. A recluse Jedi warrior, 
            he will take Luke Skywalker as apprentice .
            ''',
        ),
        Character(
            name='Dartha Vader',
            description='''
            Darth Vader is the antagonist. As a commander of the 
            evil Galactic Empire, he controls space station The 
            Death Star.
            ''',
        ),
        Character(
            name='Princess Leia',
            description='''
            Princess Leia holds the plans of the Death Star. She is 
            feisty and brave. She will become Luke's friend.
            ''',
        ),
        Character(
            name='Han Solo',
            description='''
            Han Solo is a brash mercenary space pilot of the 
            Millenium Falcon and a friend of Chebacca. He will
            take Luke on his spaceship.
            ''',
        ),
        Character(
            name='Chewbacca',
            description='''
            Chewbacca is a furry and trustful monster. He is a friend 
            of Han Solo and a copilot on the Millemium Falcon.
            ''',
        ),
    ],
    locations=[
        Location(
            name='Farm',
            description='The farm is a desert planet where Luke Skywalker lives',
        ),
    ],
    outline=[
        SceneBeat(
            location='A farm on planet Tatooine',
            plot_element='The Ordinary World',
            description='Luke Skywalker is living a normal and humble life as a farm boy on his home planet.',
        ),
        SceneBeat(
            location='Desert of Tatooine',
            plot_element='Call to Adventure',
            description='''
            Luke is called to his adventure by robot R2-D2 and Ben Kenobi. 
            Luke triggers R2-D2's message from Princess Leia and is intrigued 
            by her message. When R2-D2 escapes to find Ben Kenobi, Luke follows 
            and is later saved by Kenobi, who goes on to tell Luke about his Jedi 
            heritage. Kenobi suggests that he should come with him.
            '''
        ),
        SceneBeat(
            location="Ben Kenobi's farm",
            plot_element='Refusal of the Call',
            description='''
            Luke refuses Kenobi, telling him that he can take Kenobi and the 
            droids as far as Mos Eisley Spaceport - but he can't possibly leave 
            his Aunt and Uncle behind for some space adventure.
            ''',
        ),
        SceneBeat(
            location='A farm on planet Tatooine',
            plot_element='Crossing the First Threshold',
            description='''
            When Luke discovers that the stormtroopers searching for the droids 
            would track them to his farm, he rushes to warn his Aunt and Uncle, 
            only to discover them dead by the hands of the Empire. When Luke 
            returns to Kenobi, he pledges to go with him to Alderaan and learn 
            the ways of the Force like his father before him.
            ''',
        ),
        SceneBeat(
            location='On spaceship The Millenium Falcon',
            plot_element='Tests, Allies, Enemies',
            description='''
            After Luke, Kenobi, and the droids hire Han Solo and Chewbacca to 
            transport them onto Alderaan, Kenobi begins Luke's training in the 
            ways of the Force. Wielding his father's lightsaber, Kenobi 
            challenges Luke. At first, he can't do it. But then Kenobi tells 
            Luke to reach out and trust his feelings. Luke succeeds.
            ''',
        ),
        SceneBeat(
            location='On spaceship The Millenium Falcon',
            plot_element='Approach to the Inmost Cave',
            description='''
            The plan to defeat the Galactic Empire is to bring the Death Star 
            plans to Alderaan so that Princess Leia's father can take them to 
            the Rebellion. However, when they arrive within the system, the 
            planet is destroyed. They come across the Death Star and are pulled 
            in by a tractor beam, now trapped within the Galactic Empire.
            ''',
        ),
        SceneBeat(
            location='On spacestation The Death Star',
            plot_element='Ordeal',
            description='''
            As Kenobi goes off to deactivate the tractor beam so they can escape, 
            Luke, Han, and Chewbacca discover that Princess Leia is being held on 
            the Death Star with them. They rescue her and escape to the Millennium 
            Falcon, hoping that Kenobi has successfully deactivated the tractor 
            beam. Kenobi later sacrifices himself as Luke watches Darth Vader 
            strike him down. Luke must now avenge his fallen mentor and carry on 
            his teachings.
            ''',
        ),
        SceneBeat(
            location='On spacestation The Death Star',
            plot_element='Reward',
            description='''
            Luke has saved the princess and retrieved the Death Star plans. 
            They now have the knowledge to destroy the Galactic Empire's 
            greatest weapon once and for all.
            ''',
        ),
        SceneBeat(
            location='On spaceship The Millenium Falcon',
            plot_element='The Road Back',
            description='''
            Luke, Leia, Han, Chewbacca, and the droids are headed to the hidden 
            Rebellion base with the Death Star plans. They are suddenly pursued 
            by incoming TIE-Fighters, forcing Han and Luke to take action to 
            defend the ship and escape with their lives - and the plans. They 
            race to take the plans to the Rebellion and prepare for battle.
            ''',
        ),
        SceneBeat(
            location='On fighter ship X-Wing',
            plot_element='The Resurrection',
            description='''
            The Rebels - along with Luke as an X-Wing pilot - take on the Death 
            Star. The Rebellion and the Galactic Empire wage war in an epic space 
            battle. Luke is the only X-Wing pilot that was able to get within the 
            trenches of the Death Star. But Darth Vader and his wingmen are in hot 
            pursuit. Just as Darth Vader is about to destroy Luke, Han returns and 
            clears the way for Luke. Luke uses the Force to guide his aiming as he 
            fires upon the sole weak point of the deadly Death Star, destroying it 
            for good.
            ''',
        ),
        SceneBeat(
            location='At the Rebellion base',
            plot_element='The Return',
            description='''
            Luke and Han return to the Rebellion base, triumphant, as they receive 
            medals for the heroic journey. There is peace throughout the galaxy - at 
            least for now.
            ''',
        ),
    ],
) 
    

The first step Dramatron defines in the generation process is writing a title based on a user defined logline. Instead of writing this manually, we can use a prompt to generate the logline from a title.

In [4]:
title = 'Uncut Gems'
logline = prompt(f'write a logline for {title}')

Next, we define a function that generate an alternative title based on the logline and then use it to add a title to a story instance.

In [5]:
from promptx.template import Example

examples = [
    Example(
        star_wars.logline,
        "The Death Star's Menace"
    ),
    Example(
        "Residents of San Fernando Valley are under attack by flying saucers from outer space. The aliens are extraterrestrials who seek to stop humanity from creating a doomsday weapon that could destroy the universe and unleash the living dead to stalk humans who wander into the cemetery looking for evidence of the UFOs. The hero Jeff, an airline pilot, will face the aliens.",
        "The Day The Earth Was Saved By Outer Space."
    )
]

def write_title(story: Story) -> str:
    return prompt(
        'Suggest a alternative, original and descriptive title for a known story.',
        story.logline,
        examples=examples,
    )

story = Story(
    logline=logline,
)

title = write_title(story)
story.title = title
story


Story(
    id='09d91935-5f18-410a-9a19-1d1a27eb5f5c',
    type='story',
    logline='"An adrenaline-fueled jeweler in New York City races against time to repay his debts while juggling family, love, and a high-stakes bet that could either make him a millionaire or destroy his life."',
    title='Diamonds, Debts, and Desperation',
    outline=None,
    characters=None,
    locations=None
)

Next we create a list of character objects based on the story logline and title. We then store them as embeddings so we can query them in later steps.

In [6]:
import json

def create_characters(story: Story, n=5) -> List[Character]:
    return prompt(
        f'Create {n} characters for a story.',
        input=story.logline,
        output=[Character],
        examples=[
            Example(
                star_wars.logline,
                json.dumps([c.model_dump() for c in star_wars.characters])
            ),
        ],
    )

characters = create_characters(story)
story.characters = characters.objects
story


Story(
    id='09d91935-5f18-410a-9a19-1d1a27eb5f5c',
    type='story',
    logline='"An adrenaline-fueled jeweler in New York City races against time to repay his debts while juggling family, love, and a high-stakes bet that could either make him a millionaire or destroy his life."',
    title='Diamonds, Debts, and Desperation',
    outline=None,
    characters=[
        Character(
            id='42f4419a-7f49-4313-8241-9d4e01bf6601',
            type='character',
            name='Howard Ratner',
            description='\n            Howard Ratner is the protagonist. He is a jeweler in New \n            York City who is desperate to repay his debts and turns to \n            high-stakes gambling as a way out.\n            '
        ),
        Character(
            id='59150ef6-f036-42a2-9ae9-6b964220d7d9',
            type='character',
            name='Dinah Ratner',
            description="\n            Dinah Ratner is Howard's wife. She supports him but is \n            incre

Now we can generate a plot outline from the story logline, title, and characters.

In [7]:
def write_beats(story: Story, n=10) -> List[SceneBeat]:
    return prompt(
        f'''
        Write a sequence of {n} scene beats for a story a hero's journey structure.
        ''',
        input=dict(logline=story.logline, characters=story.characters),
        output=[SceneBeat],
        examples=[
            Example(
                json.dumps(dict(logline=star_wars.logline, characters=[c.model_dump() for c in star_wars.characters])),
                json.dumps([b.model_dump() for b in star_wars.outline]),
            ),
        ],
    )

beats = write_beats(story)
story.outline = beats.objects
story


Story(
    id='09d91935-5f18-410a-9a19-1d1a27eb5f5c',
    type='story',
    logline='"An adrenaline-fueled jeweler in New York City races against time to repay his debts while juggling family, love, and a high-stakes bet that could either make him a millionaire or destroy his life."',
    title='Diamonds, Debts, and Desperation',
    outline=[
        Scenebeat(
            id='4fc448d4-ab6a-49da-979b-1cbeea08d888',
            type='scenebeat',
            location='New York City',
            plot_element='The Ordinary World',
            description='Howard Ratner is a jeweler living in New York City, where he is struggling to repay his debts and keep his business afloat.'
        ),
        Scenebeat(
            id='eb2f9592-ec93-4c3b-a511-61f0910e9aaf',
            type='scenebeat',
            location="Howard's jewelry store",
            plot_element='Call to Adventure',
            description="When professional basketball player Kevin Garnett enters Howard's store and becom

Next we extract the scene 'beats' from the plot outline generated in the previous step. These are the main events that happen in the story.

Each scene beat has a location name so we can use this to extract location objects. These are stored like characters. For some reason the examples in the paper don't use locations from Star Wars. They also use the story logline and just the name of the location from the scene beat instead of the description of the scene. It's unclear why these decisions were made, but for consistency we'll do the same.

In [8]:
def extract_locations(story: Story) -> List[Location]:
    locations = []
    for beat in story.outline:
        response = prompt(
            '''
            Generate a location based on the story logline and location name. 
            If the location is already known, return None
            ''',
            input=dict(logline=story.logline, name=beat.location, known_locations=[l.name for l in locations]),
            output=Location,
            allow_none=True,
            examples=[
                Example(
                    json.dumps(
                        dict(
                            logline="Morgan adopts a new cat, Misterio, who sets a curse on anyone that pets them.",
                            name="The Adoption Center",
                        )
                    ),
                    Location(
                        name="The Adoption Center",
                        description='''
                        The Adoption Center is a sad place, especially for an unadopted 
                        pet. It is full of walls and walls of cages and cages. Inside of 
                        each is an abandoned animal, longing for a home. The lighting is 
                        dim, gray, buzzing fluorescent.
                        ''',
                    ).model_dump_json()
                ),
                Example(
                    json.dumps(
                        dict(
                            logline="James finds a well in his backyard that is haunted by the ghost of Sam.",
                            name="The Well",
                        ),
                    ),
                    Location(
                        name="The Well",
                        description='''
                        The well is buried under grass and hedges. It is at least 
                        twenty feet deep, if not more and it is masoned with stones. 
                        It is 150 years old at least. It stinks of stale, standing 
                        water, and has vines growing up the sides. It is narrow enough 
                        to not be able to fit down if you are a grown adult human.
                        ''',
                    ).model_dump_json()
                ),
                Example(
                    json.dumps(
                        dict(
                            logline="Mr. Dorbenson finds a book at a garage sale that tells the story of his own life. And it ends in a murder! ",
                            name="The Garage Sale",
                        )
                    ),
                    Location(
                        name="The Garage Sale",
                        description='''
                        It is a garage packed with dusty household goods and antiques. 
                        There is a box at the back that says FREE and is full of paper 
                        back books.
                        ''',
                    ).model_dump_json()
                ),
            ],
        )
        if response is not None:
            locations.append(response)
    return locations

locations = extract_locations(story)
story.locations = locations
story


Story(
    id='09d91935-5f18-410a-9a19-1d1a27eb5f5c',
    type='story',
    logline='"An adrenaline-fueled jeweler in New York City races against time to repay his debts while juggling family, love, and a high-stakes bet that could either make him a millionaire or destroy his life."',
    title='Diamonds, Debts, and Desperation',
    outline=[
        Scenebeat(
            id='4fc448d4-ab6a-49da-979b-1cbeea08d888',
            type='scenebeat',
            location='New York City',
            plot_element='The Ordinary World',
            description='Howard Ratner is a jeweler living in New York City, where he is struggling to repay his debts and keep his business afloat.'
        ),
        Scenebeat(
            id='eb2f9592-ec93-4c3b-a511-61f0910e9aaf',
            type='scenebeat',
            location="Howard's jewelry store",
            plot_element='Call to Adventure',
            description="When professional basketball player Kevin Garnett enters Howard's store and becom

Finally, we can write a script for each scene using the generated scene beats and the characters and locations we generated earlier. Characters and locations are queried using the scene beat data so that the most relevant entries are found using the stored embeddings. Again, the example follows the paper.

In [15]:
def write_script(story: Story, n=None) -> str:
    inputs = [
        dict(
            location=query(beat.location, where={'type': 'location'}).first,
            characters=query(beat.description, where={'type': 'character'}).objects,
            plot_element=story.outline[i].plot_element,
            logline=story.logline,
            previous_beat=story.outline[i-1].description if i > 0 else None,
            beat=story.outline[i].description,
        ) for i, beat in enumerate(story.outline[:n or len(story.outline)])
    ]
    
    script = []
    for input in inputs:
        page = prompt(
            '''
            Write a script based on the characters and locations
            and the events described in the scene beat.
            ''',
            input=input,
            examples=[
                Example(
                    json.dumps(
                        dict(
                            location=Location(
                                name='Cockpit of an airplane',
                                description='''
                                Cockpit of a modern passenger airplane, American Flight 812.
                                ''',
                            ).model_dump(),
                            characters=[
                                Character(
                                    name='Jeff',
                                    description='''
                                    Jeff is the hero. A man in his early forties, he tries 
                                    to stay calm in all circumstance. Jeff is now a airline 
                                    pilot.
                                    ''',
                                ).model_dump(),
                                Character(
                                    name='Danny',
                                    description='''
                                    Danny, a young airplane pilot in his thirties, is eager 
                                    to learn but can quickly lose his composture. Danny is 
                                    enamored of Edith.
                                    ''',
                                ).model_dump(),
                                Character(
                                    name='Edith',
                                    description='''
                                    Edith, an experienced stewardess with a good sense of 
                                    humour, is trustworthy and dependable. Edith likes to 
                                    tease Danny.
                                    ''',
                                ).model_dump(),
                            ],
                            plot_element=" Crossing the First Threshold",
                            logline="Residents of San Fernando Valley are under attack by flying saucers from outer space. The aliens are extraterrestrials who seek to stop humanity from creating a doomsday weapon that could destroy the universe and unleash the living dead to stalk humans who wander into the cemetery looking for evidence of the UFOs. The hero Jeff, an airline pilot, will face the aliens.",
                            previous_beat='''
                            Flight captain Jeff reluctantly leaves his wife Paula to go 
                            for a two-day flight.
                            ''',
                            beat='''
                            At the cockpit, flight captain Jeff is preoccupied by the 
                            flying saucer appearances and graveyard incidents in his home 
                            town, where he left wis wife Paula. Without success, co-pilot 
                            Danny and stewardess Edith try to reassure him.
                            ''',
                        )
                    ),
                    '''
                    DANNY
                    You're mighty silent this trip, Jeff.
                    JEFF
                    Huh?
                    DANNY
                    You haven't spoken ten words since takeoff.
                    JEFF
                    I guess I'm preoccupied, Danny.
                    DANNY
                    We've got thirty-three passengers back there that have time to 
                    be preoccupied. Flying this flybird doesn't give you that 
                    opportunity.
                    JEFF
                    I guess you're right, Danny.
                    DANNY
                    Paula?
                    JEFF
                    Yeah.
                    DANNY
                    There's nothing wrong between you two?
                    JEFF
                    Oh no, nothing like that. Just that I'm worried, she being there 
                    alone and those strange things flying over the house and those 
                    incidents in the graveyard the past few days. It's just got me 
                    worried.
                    DANNY
                    Well, I haven't figured out those crazy skybirds yet but I give 
                    you fifty to one odds the police have figured out that cemetery 
                    thing by now.
                    ( Enter EDITH )
                    JEFF
                    I hope so.
                    EDITH
                    If you're really that worried Jeff why don't you radio in and 
                    find out? Mac should be on duty at the field by now. He could call 
                    Paula and relay the message to you.
                    DANNY
                    Hi Edith.
                    EDITH
                    Hi Silents. I haven't heard a word from this end of the plane since 
                    we left the field.
                    DANNY
                    Jeff's been giving me and himself a study in silence.
                    EDITH
                    You boys are feudin'?
                    JEFF
                    Oh no Edie, nothing like that.
                    DANNY
                    Hey Edie, how about you and me balling it up in Albuquerque?
                    EDITH
                    Albuquerque? Have you read that flight schedule Boy?
                    DANNY
                    What about it?
                    EDITH
                    We land in Albuquerque at 4 am. That's strictly a nine o'clock 
                    town.
                    DANNY
                    Well I know a friend that ll help us --
                    EDITH
                    Let's have a problem first, huh Danny.
                    DANNY
                    Ah he's worried about Paula.
                    EDITH
                    I read about that cemetery business. I tried to get you kids to 
                    not buy too near one of those things. We get there soon enough as 
                    it is.
                    DANNY
                    He thought it'd be quiet and peaceful there.
                    EDITH
                    No doubt about that. It's quiet alright, like a tomb. I'm sorry 
                    Jeff, that was a bad joke.
                    '''
                )
            ]
        )
        script.append(page)
    return script


script = write_script(story)
script


[
    "HOWARD\n    I can't believe this mess I'm in. The debts keep getting bigger and my business is going under.\n    \nJULIA\n    Howard, you need to find a way to repay those debts. Maybe we can sell some of the jewelry in your store.\n    \nHOWARD\n    It's not enough, Julia. I need something big, something valuable that will make me a fortune.\n    \nDEMAN\n    I might know a guy who can help you out, Howard. He's got connections in the underworld and might have a job for you.\n    \nHOWARD\n    Anything, Deman. I'll do anything to get out of this mess.\n    \nARNO\n    You better make it happen, Ratner. I don't have time for your games.\n    \nHOWARD\n    I understand, Arno. I'll get you the money, I promise.\n    \nDINAH\n    Howard, be careful dealing with Arno. He's not someone you want to mess with.\n    \nHOWARD\n    I know, Dinah. But I'm desperate. I need this opportunity to turn my life around.",
    "DINAH \n        So Howard, you've got yourself in quite the predicame

Now we have defined all the functions, lets put it together in a single cell and generate a new story.

The data we've generated only exists in memory.